# Sequências - Aula Prática 01/04
## Embeddings

Neste notebook iremos trabalhar um pouco com a parte introdutória de modelos de linguagem, realizando *analogias* a partir de representações distribuídas (*embeddings*) geradas pelo algoritmo de aprendizagem não-supervisionada `GloVe` (*Global Vectors for Word Representation*).
> Para saber mais sobre como o `GloVe` funciona, recomendo a leitura do seguinte [notebook](https://colab.research.google.com/github/jaygala24/pytorch-implementations/blob/master/Global%20Vectors%20for%20Word%20Representation.ipynb#scrollTo=oEXgG-hDIMdT). Esse notebook prático não tem como foco explicar como que o algoritmo funciona, mas sim explicar as operações que podemos fazer com as representações distribuídas geradas por algoritmos de *word embedding*.

- Este notebook foi inspirado pelos trabalhos disponibilizados nos sites [d2l.ai](https://d2l.ai/chapter_natural-language-processing-pretraining/similarity-analogy.html) e [notebook.community](https://notebook.community/spro/practical-pytorch/glove-word-vectors/glove-word-vectors).

- **Importante:** caso esteja rodando esse notebook no ambiente da Tatu, favor descomentar e executar a seguinte célula. Caso contrário, basta deixá-la comentada e ignorar a sua execução.

In [ ]:
# %load_ext nbproxy

Definindo um diretório temporário para salvar dados e eventuais saídas de modelos.

In [ ]:
import tempfile
tmp = tempfile.TemporaryDirectory()

print('Nome do diretório temporário:', tmp.name)

## Importação de pacotes

In [ ]:
import torch
import torchtext

import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim

import seaborn as sns
import matplotlib.pyplot as plt

from os.path import join as ospj
from torch.utils.data import Dataset

sns.set_style('dark')

In [ ]:
# Definindo se o código será executado na CPU ou na GPU
has_cuda = torch.cuda.is_available()
device = torch.device('cuda' if has_cuda else 'cpu')

print('O código será executado em:', device)

## Analogias

Analogias são associações de mesma natureza entre palavras (como flexões de gênero ou número). A geometria dessas associações pode ser visualizada no espaço vetorial onde as palavras são projetadas e, em modelos bem treinados, deve ser possível encontrar semelhanças entre associações de mesma natureza.

<img width=600 src="../imagens/analogias.png"/>

Ao longo deste notebook, utilizaremos o modelo `GloVe` para realizar a projeção de palavras em um espaço vetorial. Mais especificamente, usaremos o módulo `torchtext` para carregar um modelo pré-treinado em uma base de dados de 6 bilhões de tokens, realizando uma projeção das palavras em um espaço 100 dimensional.

In [ ]:
# Aqui temos uma lista de modelos pré-treinados que estão disponíveis na torchtext
# No caso do GloVe, a nomenclatura utilizada é: glove.<tamanho-do-corpus>.<dimensão>
torchtext.vocab.pretrained_aliases.keys()

### Carregamento de um modelo GloVe pré-treinado

Iremos utilizar a classe `torchtext.vocab.GloVe` para carregar um modelo `GloVe` pré-treinado.

> O modelo retornado pelo `torchtext` possui alguns atributos, como o dicionário `stoi` (*string* to *int*) para mapear uma palavra para um índice numérico e a lista `itos` (*int* to *string*), mapeando um índice númerico para uma palavra. Além disso, conseguimos acessar a matriz de *embeddings* através do atributo `vectors`.

In [ ]:
glove = torchtext.vocab.GloVe(name='6B', dim=100, cache=tmp.name)

random_tokens = np.random.choice(glove.itos, size=5)

print('Número de tokens mapeados pelo glove.6B.100d:', len(glove.stoi))
print('Aqui temos 5 tokens aleatórios mapeados pelo glove.6B.100d:', random_tokens)
print('A dimensionalidade da matriz de embeddings é:', glove.vectors.shape)

### Projeções vetoriais utilizando o GloVe

Podemos realizar projeções de palavras utilizando a matriz de *embeddings* computada pelo `glove.6B.100d` através do método `get_vecs_by_tokens`, como visto a seguir.

In [ ]:
tokens = ['man', 'king', 'woman', 'queen']
vecs = glove.get_vecs_by_tokens(tokens)

plt.matshow(vecs, aspect='auto')
plt.yticks(range(4), tokens)
plt.colorbar()
plt.show()

Além disso, podemos visualizar esses vetores no espaço latente para analisar algumas propriedades interessantes. Para tornar a visualização factível, podemos utilizar um algoritmo de redução de dimensionalidade.

> Para esse exemplo iremos utilizar o algoritmo de redução de dimensionalidade PCA (*Principal Components Analysis*), devido a uma melhor interpretabilidade para o exemplo escolhido. Porém, é mais comum vermos algoritmos como o t-SNE (*t-distributed Stochastic Neighbor Embedding*), uma vez que este algoritmo preserva a estrutura local do dado original. Para mais detalhes na diferença entre t-SNE e PCA, recomendo a leitura do seguinte [link](https://medium.com/analytics-vidhya/pca-vs-t-sne-17bcd882bf3d).

In [ ]:
from sklearn.decomposition import PCA

tokens = ['man', 'woman', 'king', 'queen', 'brother', 'sister']
vecs = glove.get_vecs_by_tokens(tokens)

# Calculando a redução de dimensionalidade para um espaço bidimensional
pca = PCA(n_components=2)
components = pca.fit_transform(vecs.numpy())

print('Tamanho dos componentes:', components.shape)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(components[:, 0], components[:, 1])

for i, token in enumerate(tokens):
    ax.annotate(token, (components[i, 0], components[i, 1]))

for i in range(0, len(tokens) - 1, 2):
    x = (components[i, 0], components[i+1, 0])
    y = (components[i, 1], components[i+1, 1])
    ax.plot(x, y, linestyle='--')

Note pela imagem acima como a operação vetorial: $(\overrightarrow{\text{king}} - \overrightarrow{\text{queen}}) + \overrightarrow{\text{sister}} \approx \overrightarrow{\text{brother}}$.

In [ ]:
king = components[tokens.index('king')]
queen = components[tokens.index('queen')]
sister = components[tokens.index('sister')]
brother = components[tokens.index('brother')]

result = (king - queen) + sister
print('Vetor resultante:', result)
print('Vetor para a palavra "brother":', brother)

<br> Podemos refazer o plot para verificar onde a posição do vetor resultante.

In [ ]:
fig, ax = plt.subplots()

# Adicionando o ponto do vetor resultante
ax.scatter(result[0], result[1], color='red')
ax.annotate('resultante', (result[0], result[1]))

ax.scatter(components[:, 0], components[:, 1])

for i, token in enumerate(tokens):
    ax.annotate(token, (components[i, 0], components[i, 1]))

for i in range(0, len(tokens) - 1, 2):
    x = (components[i, 0], components[i+1, 0])
    y = (components[i, 1], components[i+1, 1])
    ax.plot(x, y, linestyle='--')

### Realizando analogias através do GloVe

Através de operações vetoriais, como visto anteriormente, somos capazes de criar analogias no espaço latente da nossa representação distribuída, conseguindo responder perguntas como: `rainha está para rei assim como irmã está para ???`.

Possuímos diversas estratégias para retornar a palavra correta para tal pergunta. Por exemplo, a partir da imagem da célula anterior, uma possível estratégia é de calcular o top-1 vizinho mais próximo do vetor resultante através de um algoritmo de `KNN`, porém tal estratégia pode ser menos viável em dimensões maiores devido à [maldição da dimensionalidade](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=6065061). Para esse notebook, iremos utilizar uma estratégia que envolve a similaridade de cosseno entre os vetores no espaço latente, selecionando como analogia aquele vetor com maior similaridade.

A similaridade de cosseno pode ser computada através da seguinte equação, onde temos uma operação de similaridade no numerador (produto interno), dividido pelo produto da norma de cada vetor. Também introduzimos um fator $\epsilon \approx 1\mathrm{e}{-6}$ no denominador como um termo para evitar divisão por zero. A distância, ou similaridade, de cosseno possui imagem entre $[-1, +1]$, onde $-1$ indica que os vetores estão apontados para sentidos opostos; $+1$ indica que os vetores estão apontando para a mesma direção; e $0$ indica que os vetores são perpendiculares.

$$
\text{similaridade} = \frac{\overrightarrow{x_1} \cdot \overrightarrow{x_2}}{\max(\lVert \overrightarrow{x_1} \rVert_2 * \lVert \overrightarrow{x_2} \rVert_2, \epsilon)}
$$

Primeiramente, iremos observar os top vetores retornados pela distância de cosseno a partir de uma *query* (palavra) fornecida.

In [ ]:
def get_similar_tokens(query, embedding, k=10):
    query_vector = embedding.get_vecs_by_tokens(query)
    query_vector = query_vector.unsqueeze(dim=0)  # adicionando uma dimensão para casar com as dimensões do embeddings
    
    cosine = nn.CosineSimilarity(dim=1)
    similarities = cosine(query_vector, embedding.vectors)

    result = []
    values, indices = torch.topk(similarities, k=k+1)  # a query será mais similar com ela mesma, por isso iremos usar k+1
    for value, index in zip(values[1:], indices[1:]):
        token = embedding.itos[index]
        result.append((token, value))

    return result

similar_tokens = get_similar_tokens('computer', glove, k=5)
for i, (token, similarity) in enumerate(similar_tokens):
    print(f'{i+1}. {token} -> {similarity:.5f}')

<br> Agora iremos utilizar a distância de cosseno para computar analogias.

In [ ]:
def get_analogy(token1, token2, token3, embedding):
    """
    A ordem dos tokens passados é: <token1> está para <token2> assim como <token3> está para...
    """
    vecs = embedding.get_vecs_by_tokens([token1, token2, token3])
    result = (vecs[1] - vecs[0]) + vecs[2]
    result = result.unsqueeze(dim=0)  # adicionando uma dimensão para casar com as dimensões do embeddings

    cosine = nn.CosineSimilarity(dim=1)
    similarities = cosine(result, embedding.vectors)
    analogy = embedding.itos[similarities.argmax()]

    return analogy

get_analogy('queen', 'king', 'sister', glove)

In [ ]:
get_analogy('beijing', 'china', 'tokyo', glove)

In [ ]:
get_analogy('do', 'did', 'go', glove)

## Extra) Utilizando um modelo pré-treinado para uma tarefa de analogias

Nessa parte opcional do notebook, iremos ver como podemos usar um modelo pré-treinado de representações distribuídas, como o caso do `GloVe`, e realizar um *fine-tuning* em uma base de dados específica.

> No nosso caso, o `GloVe` já é um modelo poderoso o suficiente para realizarmos essas analogias, então essa seção será apenas para ilustrar o procedimento necessário para carregar os pesos de um modelo pré-treinado e como realizar o *fine-tuning* da camada de *embedding*.

Antes de continuar com essa parte, iremos carregar os dados e definir um `Dataset` PyTorch customizado.

In [ ]:
!wget -P "$tmp.name" https://www.dropbox.com/s/f8k3xoywff0h3br/questions-words.csv

In [ ]:
df = pd.read_csv(ospj(tmp.name, 'questions-words.csv'))
df

In [ ]:
class AnalogiesDataset(Dataset):
    def __init__(self, dataframe: pd.DataFrame):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx: int):
        df_line = self.dataframe.iloc[idx]

        # Processando os inputs
        inputs = df_line['input'].lower()  # deixando o texto em minúsculo
        inputs = inputs.split(' ')  # tokenização simples por palavra

        # Processando as labels
        labels = df_line['analogy'].lower()  # deixando o texto em minúsculo
        
        return inputs, labels

dataset = AnalogiesDataset(df)
print('Tamanho do conjunto de dados:', len(dataset))
print('Primeira amostra do conjunto de dados:', dataset[0])

Agora iremos definir o nosso modelo para realizar as analogias. A idéia por trás desse modelo é de utilizar uma camada de *embedding* pré-treinada e utilizar uma camada linear para realizar uma combinação dos vetores de entrada (cálculo da analogia), e indicar qual palavra é mais provável de ser a resposta para a nossa analogia.

> Fundamentalmente, estaremos trabalhando com um problema de classificação, onde queremos classificar qual é o *token* mais provável de ser a resposta da nossa analogia. Para isso, utilizaremos como função de perda uma entropia cruzada.

In [ ]:
class AnalogyNet(nn.Module):
    def __init__(self, vocab_size: int, embedding_dim: int, embedding_weights: torch.Tensor = None):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(embedding_dim, vocab_size)

        # Inicializando os pesos da matriz de embedding
        if embedding_weights is not None:
            self.embedding.weight.data.copy_(embedding_weights)

    def forward(self, x: torch.Tensor):
        x = self.embedding(x)
        x = torch.sum(x, dim=0)  # sumarizando os embeddings em um tensor unidimensional
        return self.linear(x)

Com os dados e modelo prontos, conseguimos finalmente realizar o *fine-tuning* do nosso modelo.

In [ ]:
from tqdm.notebook import tqdm

num_epochs = 50
batch_size = 64
learning_rate = 0.001

model = AnalogyNet(vocab_size=len(glove.stoi), embedding_dim=100, embedding_weights=glove.vectors)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

dataset = AnalogiesDataset(df)

for epoch in range(num_epochs):
    total_loss = 0
    
    for X, y in tqdm(dataset):
        # Temos que converter os dados de strings para tensores numéricos
        # Para isso, utilizaremos o mapeamento do GloVe, uma vez a nossa matriz de embeddings
        # segue a mesma ordem da matriz de embeddings do modelo pré-treindo
        X = torch.LongTensor([glove.stoi[token] for token in X])
        y = torch.tensor(glove.stoi[y])

        X = X.to(device)
        y = y.to(device)

        outputs = model(X)
        total_loss += criterion(outputs, y)

    optimizer.zero_grad()
    total_loss /= len(dataset)
    total_loss.backward()
    optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs} => mean loss: {total_loss:.5f}')

In [ ]:
# get_analogy('athens', 'greece', 'bern', glove)  # resposta correta: Suíça